In [93]:
import collections

class NTA:
    def __init__(self, input_file):
        self.name = ''
        self.states = ''
        self.alphabet = ''
        self.tape = ''
        self.start = ''
        self.accept = ''
        self.reject = ''
        self.transitions = []
        self.parse_input(input_file)

    def parse_input(self, input_file):
        sections = []
        with open(input_file, 'r') as opened_file:
            for line in opened_file:
                sections.append([x for x in line.strip().split(',') if x])

        self.name = sections[0]
        self.states = sections[1]
        self.alphabet = sections[2]
        self.tape = sections[3]
        self.start = sections[4][0]
        self.accept = sections[5]
        self.reject = sections[6]
        self.transitions = sections[7:]

    def decompose_transition(self):
        dictionary = collections.defaultdict(list)
        for state in self.transitions:
            if len(state[0]) == 2 and state[0] in self.states:
                dictionary[state[0]].append(state[1:])
        return dictionary

class TuringMachineSimulator:
    def __init__(self, nta):
        self.nta = nta

    def make_tree(self, string, states_map):
        tree = [[[self.nta.start, -1, -1, string, 0]]]
        depth = -1
        while depth < 15:
            depth += 1
            current_level = []
            next_node = 'na'
            for track_pindex, state in enumerate(tree[-1]):
                node, pheight, pindex, string, strindex = state
                for child in states_map[node]:
                    if strindex >= len(string):
                        string += '_'
                    if child[0] == string[strindex]:  # Match
                        temp = string
                        string = list(string)
                        string[strindex] = child[2] if len(child) >= 3 else string[strindex]
                        string = ''.join(string)
                        direction = 1 if len(child) >= 3 and child[3] == 'R' else -1
                        next_node = child[1]
                        current_level.append([next_node, len(tree) - 1, track_pindex, string, strindex + direction])
                        string = temp
                    else:
                        current_level.append([self.nta.reject[0], len(tree) - 1, track_pindex, string, strindex])
            tree.append(current_level)
            if tree[-1] == []:
                return self.nta.reject
            if next_node == self.nta.accept[0]:
                return tree, self.nta.accept[0]
            elif next_node == 'na':
                return tree, self.nta.reject[0]
        return tree

    def path_len(self, tree):
        path = []
        flag = False
        height, index = tree[-1][-1][1], tree[-1][-1][2]
        for level in tree[::-1]:
            for state in level:
                if state[0] == self.nta.accept[0]:
                    height, index = state[1], state[2]
                    path.append(self.nta.accept[0])
                    flag = True
                    break
            if flag:
                break
        while height != -1 and index != -1:
            path.append(tree[height][index][0])
            height, index = tree[height][index][1], tree[height][index][2]
        return path[::-1]

    def configure_output(self, tree):
        path = []
        flag = False
        ans = []
        height, index = tree[-1][-1][1], tree[-1][-1][2]
        for level in tree[::-1]:
            for state in level:
                if state[0] == self.nta.accept[0]:
                    height, index = state[1], state[2]
                    ans.append(state[3][:state[4]] + f'[{state[0]}]' + state[3][state[4]:])
                    path.append(self.nta.accept[0])
                    flag = True
                    break
            if flag:
                break
        while height != -1 and index != -1:
            head = tree[height][index][4]
            curstring = tree[height][index][3]
            curstate = tree[height][index][0]
            if head == 0:
                output = f'[{curstate}]' + curstring
            else:
                output = curstring[:head] + f'[{curstate}]' + curstring[head:]
            ans.append(output)
            path.append(tree[height][index][0])
            height, index = tree[height][index][1], tree[height][index][2]
        return ans

    def calc_trans(self, tree):
        size = 0
        for level in tree:
            for node in level:
                size += 1
        return size

def main():
    input_file = "addition.csv"
    string = 'aabbcc'

    nta = NTA(input_file)
    simulator = TuringMachineSimulator(nta)
    states_map = nta.decompose_transition()
    tree, status = simulator.make_tree(string, states_map)

    size = simulator.calc_trans(tree)
    print(nta.name[0])
    print(string)
    print(f'{size - 2} total transitions traced')
    path = simulator.path_len(tree)
    print(f'String accepted in {len(path) - 1}' if status == nta.accept[0] else f'String rejected in {len(path) - 1}')
    ans = simulator.configure_output(tree)
    for s in ans[::-1]:
        print(s)

if __name__ == '__main__':
    main()


IndexError: list index out of range